# Build (Targets)
Builds the `targets` table from a flat file database using names as the primary key which are assumed to be unique strings for this project. Other notebooks should be used for adding additional records to the table.

### Import libraries

In [1]:
# Basic stack
from datetime import datetime

# Web Stack
import json
import requests

# Database stack
import sqlite3

# Data Science stack
import pandas as pd

### User defined variables

In [2]:
db = "../../db.sqlite3"
targets = "../../data/databases/targets.xlsx"

### User defined functions

In [3]:
def update_targets(c, name, column_name, column_value):
    """ Updates EE table values of a given column with a given value using the 
            ENTITY ID as the selection criteria for the update.

        C - A cursor
        ENTITY ID - An entity id from EarthExplorer
        COLUMN NAME - A column name within the EE table to be updated
        COLUMN VALUE - A value to update the column
    """
    sql_string = f"UPDATE targets SET {column_name} = \"{column_value}\" WHERE target = \"{name}\""
    c.execute(sql_string)


def database_activity(db, target, column_name, column_value, activity='update'):
    """ Wrapper function for activities related to database management.
            Currently only supports UPDATING based on EarthExplorer 
            Entity ID values.

        Is dependent on the UPDATE EE function above.

        DB - A database target
        ENTITY ID - An EarthExplorer Entity ID uniquely identifying
            the record of interest.
        COLUMN NAME - The column to be updated
        COLUMN VALUE - The value of the column to be updated
        ACTIVITY - The activity to be carried out. Currently only
            supports UPDATE.
    """
    conn = sqlite3.connect(db)
    conn.enable_load_extension(True)
    conn.execute("SELECT load_extension('mod_spatialite')")
    c = conn.cursor()

    if activity == 'update':
        try:
            update_targets(c, target, column_name, column_value)
        except Exception as e:
            print("Exception: {} was raised for {}".format(e, row['target']))
    else:
        print("Your activity method is not currently supported.")
    
    conn.commit()
    conn.close()

def insert_person(c, target):
    """ Inserts People into the People Data Table. People's 
            names are the Primary Key for the datatable since
            they are assumed to be unique for this effort.

        C - A cursor
        PERSON - A Person's Name
    """
    sql_string = f"INSERT INTO targets(target) VALUES (\"{target}\")"
    c.execute(sql_string)

### Load data

In [4]:
df = pd.read_excel(targets)
df.head()

,target,scientific_name
0,Bowhead whale,Balaena mysticetus
1,Bryde's whale,Balaenoptera brydei
2,Common eiders,Somateria mollissima
3,Fin whale,Balaenoptera physalus
4,Gray whale,Eschrichtius robustus


### Drop table
This is for demonstration purposes

In [5]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS targets''')
conn.commit()
conn.close()

### Create `targets` table

In [6]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS targets(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        target VARCHAR(30),
        scientific_name VARCHAR(25)
    )
''')

conn.commit()
conn.close()

### Insert targets into `targets` table

In [7]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

for i, row in df.iterrows():
    try:
        insert_person(c, row['target'])
    except Exception as e:
        print("Exception: {} was raised for {}".format(e, row['target']))
print("Done inserting targets into table!")
conn.commit()
conn.close()

Done inserting targets into table!


### Update `target` records from spreadsheet

In [8]:
columns = df.columns[1:]
for i, row in df.iterrows():
    tid = row['target']
    print("Updating information for: {}".format(row['target']))
    for column in columns:
        database_activity(db, tid, column, row[column])

Updating information for: Bowhead whale
Updating information for: Bryde's whale
Updating information for: Common eiders
Updating information for: Fin whale
Updating information for: Gray whale
Updating information for: Hawaiian monk seal
Updating information for: Humpback whale
Updating information for: Blue whale
Updating information for: North Atlantic right whale
Updating information for: North Pacific right whale
Updating information for: Seal
Updating information for: Southern right whale
Updating information for: Sperm whale
Updating information for: Target buoy weather station
Updating information for: Omura whale
Updating information for: Turtle
Updating information for: Rice's whale
Updating information for: Killer whale


### Select new information, review it

In [9]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

df = pd.read_sql_query(f"SELECT * FROM targets", conn)

conn.commit()
conn.close()

df.head()

,id,target,scientific_name,bbox
0,1,Bowhead whale,Balaena mysticetus,None
1,2,Bryde's whale,Balaenoptera brydei,None
2,3,Common eiders,Somateria mollissima,None
3,4,Fin whale,Balaenoptera physalus,None
4,5,Gray whale,Eschrichtius robustus,None


# End